In [34]:
import numpy as np

In [46]:
# Personas
LIMITE_INFERIOR_DEPOSITOS = 10
LIMITE_SUPERIOR_DEPOSITOS = 110
LIMITE_INFERIOR_EXTRACCIONES = 3
LIMITE_SUPERIOR_EXTRACCIONES = 50
MEDIA_TIEMPO_DEPOSITOS = 5 # 5 minutos
MEDIA_TIEMPO_EXTRACCIONES = 1.5 # 90 segundos = 1.5 ,minutos

In [47]:
# Cajero Estandar
CAPACIDAD_BILLETES_DEPOSITOS= 2000
CAPACIDAD_BILLETES_EXTRACCIONES = 2000
VALOR_BILLETES = 100

In [48]:
# Cajero Reciclador
CAPACIDAD = 2000
CANTIDAD_BILLETES_INCIAL = 1000

In [49]:
class CajeroEstandar:
    def __init__(self, capacidad_billetes_depositos, capacidad_billetes_extracciones):
        # Comienza el día con la carga completa para extracciones
        self.billetes_extracciones = capacidad_billetes_extracciones
        
        # Comienza el día sin ningún depósito
        self.capacidad_billetes_depositos = capacidad_billetes_depositos
        self.billetes_depositados = 0
    
    def depositar(billetes):    
        futuros_billetes_depositados  = self.billetes_depositados + billetes
        if futuros_billetes_depositados > self.capacidad_billetes_depositos:
            return False          
        self.billetes_depositados = futuros_billetes_depositados
        return True
    
    def extraer(billetes):
        futuros_billetes_extracciones = self.billetes_extracciones - billetes
        if futuros_billetes_extracciones < 0:
            return False
        self.billetes_extracciones = futuros_billetes_extracciones
        return True

In [50]:
class cajeroReciclador:
    def __init__(self, capacidad, cantidad_billetes_inicial):
        self.capacidad = capacidad
        self.billetes = cantidad_billetes_inicial
        
    def depositar(billets):
        futuros_billetes  = self.billetes + billetes
        if futuros_billetes > self.capacidad:
            return False
        self.billetes = futuros_billetes
        return True
    
    def extraer (billetes):
        futuros_billetes = self.billetes - billetes
        if futuros_billetes < 0:
            return False
        self.billetes = futuros_billetes
        return True

In [51]:
class Operacion:
    def __init__(tiempo, resultado):
        self.tiempo = tiempo
        self.resultado = resultado

In [52]:
class Persona():
    def __init__(limite_inferior_depositos, 
                 limite_superior_depositos, 
                 limite_inferior_extracciones,
                 limite_superior_extracciones,
                 media_tiempo_depositos,
                 media_tiempo_extracciones):
    
        self.limite_inferior_depositos = limite_inferior_depositos 
        self.limite_superior_depositos = limite_superior_depositos 
        self.limite_inferior_extracciones = limite_inferior_extracciones,
        self.limite_superior_extracciones = limite_superior_extracciones ,
        self.media_tiempo_depositos = media_tiempo_depositos,
        self.media_tiempo_extracciones = media_tiempo_extracciones
        
        
    def realizar_transaccion(cajero):
        random = np.random.uniform(0, 1, 1)[0]
        if random < 0.75:
            return self.realizar_deposito(cajero)
        return self.realizar_extraccion (cajero)
    
    def realizar_deposito (cajero):
        billetes = np.random.uniform(self.limite_inferior_depositos, self.limite_superior_depositos, 1)[0]
        tiempo = np.random.exponential(self.media_tiempo_depositos)
        resultado = cajero.depositar(billetes)
        return Operacion(tiempo, resultado)
        
    def realizar_extraccion (cajero):
        billetes = np.random.uniform(limite_inferior_extracciones, limite_superior_extracciones, 1)[0]
        tiempo = np.random.exponential(self.media_tiempo_extracciones)
        resultado =  cajero.extraer(billetes)
        return Operacion(tiempo, resultado)